In [3]:
### Set up libraries
from __future__ import division

import os
import urllib, cStringIO

import numpy as np
import scipy.stats as stats
import pandas as pd
import json
import re

from PIL import Image
import base64
import datetime
import time
import couchdb
from svgpathtools import parse_path

In [ ]:
## needed for rendering svg drawings
from matplotlib.path import Path
import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

import matplotlib.patches as patches

In [4]:
# set vars -- need access to couchdb
auth = pd.read_csv('auth.txt', header = None) 
server_address_input = auth.values[0][0]

## get our main database
couch = couchdb.Server(server_address_input)
db = couch['tsinghua-draw'] 
# db.info()

### Can use mongo-like queries on couchdb! wow! major win.

In [6]:
image_recs = {'selector': {'dataType':'finalImage', 'subID':'iPad 2_THU9M2'}}
image_recs_test = db.find(image_recs)

### lets try getting all strokes for a single category/participant

In [8]:
example_sub = 'Ipad2_THU9M23'
stroke_recs_query = {'selector': {'dataType':'stroke', 'subID': example_sub, 'category':'bike'}}
stroke_recs = db.find(stroke_recs_query)
len(stroke_recs)

9

In [10]:
def get_d_string(svg_string):
    start_ind = svg_string.find('d="')
    end_ind = svg_string.find('fill')
    to_parse = svg_string[start_ind:end_ind]
    return(to_parse)

def polyline_pathmaker(lines):
    x = []
    y = []

    codes = [Path.MOVETO] # start with moveto command always
    for i,l in enumerate(lines):
        for _i,_l in enumerate(l):
            x.append(_l[0])
            y.append(_l[1])
            if _i<len(l)-1:
                codes.append(Path.LINETO) # keep pen on page
            else:
                if i != len(lines)-1: # final vertex
                    codes.append(Path.MOVETO)
    verts = zip(x,y) 
    return verts, codes

def get_verts_and_codes(svg_list):
    '''
    parse into x,y coordinates and output list of lists of coordinates
    
    '''    
    lines = []
    Verts = []
    Codes = []
    for stroke_ind,stroke in enumerate(svg_list):
        x = []
        y = []
        to_parse = get_d_string(stroke)
        parsed = parse_path(to_parse)
        for i,p in enumerate(parsed):
            if i!=len(parsed)-1: # last line segment
                x.append(p.start.real)
                y.append(p.start.imag)    
            else:
                x.append(p.start.real)
                y.append(p.start.imag)     
                x.append(p.end.real)
                y.append(p.end.imag)
        lines.append(zip(x,y))
        verts, codes = polyline_pathmaker(lines)
        Verts.append(verts)
        Codes.append(codes) 
    return Verts, Codes

In [36]:
def render_and_save(Verts,
                    Codes,
                    line_width=5,
                    imsize=6,
                    canvas_size=809,
                    session_id='SESSION_ID',
                    kidName='KIDNAME',
                    trial_num='TRIAL_NUM',
                    category='CATEGORY'):
    
    '''
    input: 
        line_width: how wide of strokes do we want? (int)
        imsize: how big of a picture do we want? (setting the size of the figure) 
        canvas_size: original canvas size on tablet?
        out_path: where do you want to save your images? currently hardcoded below.        
    output:
        rendered sketches into nested directories
    
    '''
    ## where do you want to save your cumulative drawings?
    out_path = os.path.join('./cumulative_drawings','{}_{}'.format(kidName,session_id),'{}_{}'.format(trial_num,category))
    if not os.path.exists('./cumulative_drawings'):
        os.makedirs('./cumulative_drawings')
    if not os.path.exists(os.path.join('cumulative_drawings','{}_{}'.format(kidName,session_id))):
        os.makedirs(os.path.join('cumulative_drawings','{}_{}'.format(kidName,session_id)))

    verts = Verts[0]
    codes = Codes[0]
    for i,verts in enumerate(Verts):
        codes = Codes[i]
        fig = plt.figure(figsize=(imsize,imsize), frameon=False)    
        ax = plt.subplot(111)
        ax.axis('off')
        ax.set_xlim(0,canvas_size)
        ax.set_ylim(0,canvas_size)
        
        # remove padding for xaxis and y axis
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        # remove further paddings
        plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
            hspace = 0, wspace = 0)
        
        ### render sketch so far
        if len(verts)>0:
            path = Path(verts, codes)
            patch = patches.PathPatch(path, facecolor="none", edgecolor='black', lw=line_width)
            ax.add_patch(patch)
            plt.gca().invert_yaxis() # y values increase as you go down in image
#             plt.show()


        ## save out as png 
        ## maybe to make it not render every single thing, use plt.ioff
        if not os.path.exists(out_path):
            os.makedirs(out_path)
        fname = '{}_{}_{}_{}.png'.format(session_id,trial_num,category,i)
        filepath = os.path.join(out_path,fname)
        print filepath
        
        fig.savefig(filepath, bbox_inches='tight', pad_inches=0.0)
        plt.close(fig)        

### just test one kid that we got earlier

In [37]:
verts, codes = get_verts_and_codes(svg_list)
render_and_save(verts, codes)

./cumulative_drawings/KIDNAME_SESSION_ID/TRIAL_NUM_CATEGORY/SESSION_ID_TRIAL_NUM_CATEGORY_0.png
./cumulative_drawings/KIDNAME_SESSION_ID/TRIAL_NUM_CATEGORY/SESSION_ID_TRIAL_NUM_CATEGORY_1.png
./cumulative_drawings/KIDNAME_SESSION_ID/TRIAL_NUM_CATEGORY/SESSION_ID_TRIAL_NUM_CATEGORY_2.png
./cumulative_drawings/KIDNAME_SESSION_ID/TRIAL_NUM_CATEGORY/SESSION_ID_TRIAL_NUM_CATEGORY_3.png
./cumulative_drawings/KIDNAME_SESSION_ID/TRIAL_NUM_CATEGORY/SESSION_ID_TRIAL_NUM_CATEGORY_4.png
./cumulative_drawings/KIDNAME_SESSION_ID/TRIAL_NUM_CATEGORY/SESSION_ID_TRIAL_NUM_CATEGORY_5.png
./cumulative_drawings/KIDNAME_SESSION_ID/TRIAL_NUM_CATEGORY/SESSION_ID_TRIAL_NUM_CATEGORY_6.png
./cumulative_drawings/KIDNAME_SESSION_ID/TRIAL_NUM_CATEGORY/SESSION_ID_TRIAL_NUM_CATEGORY_7.png
./cumulative_drawings/KIDNAME_SESSION_ID/TRIAL_NUM_CATEGORY/SESSION_ID_TRIAL_NUM_CATEGORY_8.png


In [ ]:
#    if stroke_recs.count()>0:

#                 ## some handy metadata for writing out the sketch PNG filename
#                 trial_num = imrec['trialNum']
#                 category = imrec['category']
#                 session_id = imrec['sessionId']
#                 kidName = imrec['kidName']
#                 print(kidName)

#                 ## now make an svg list!
#                 svg_list = make_svg_list(stroke_recs)

#                 ## now get me some verts and codes!
#                 Verts, Codes = get_verts_and_codes(svg_list)

#                 ## now render out your cumulative sketches and save out as pngs!
#                 render_and_save(Verts,
#                                 Codes,
#                                 line_width=5,
#                                 imsize=6,
#                                 canvas_size=809,
#                                 session_id=session_id,
#                                 kidName=kidName,
#                                 trial_num=trial_num,
#                                 category=category)